In [ ]:
import os
import shutil
import random
import torch.nn as nn  # Import the nn module from torch
from sklearn.model_selection import train_test_split
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Define paths
dataset_path = "/content/drive/MyDrive/SC_190104027_Assignment1/Input" # Total Data path
train_output_path = "/content/drive/MyDrive/SC_190104027_Assignment1/Output/Train" # Train data path
test_output_path = "/content/drive/MyDrive/SC_190104027_Assignment1/Output/Test" # test data path
val_output_path = "/content/drive/MyDrive/SC_190104027_Assignment1/Output/Validation" # validation data path

# Create output directories if they don't exist
os.makedirs(train_output_path, exist_ok=True)
os.makedirs(test_output_path, exist_ok=True)
os.makedirs(val_output_path, exist_ok=True)

# List all the class directories in the dataset path
class_dirs = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]

# Shuffle the class directories to ensure randomness
random.shuffle(class_dirs)

# Split percentages
train_percentage = 0.7
test_percentage = 0.2
val_percentage = 0.1


# Loop through each class directory and copy files to appropriate splits
for class_dir in class_dirs:
    files = os.listdir(os.path.join(dataset_path, class_dir))
    random.shuffle(files)
    #print(files)

    total_samples = len(files)
    train_samples = int(train_percentage * total_samples)

    test_samples = int(test_percentage * total_samples)

    val_samples = total_samples - train_samples - test_samples


    train_files = files[:train_samples]

    test_files = files[train_samples:train_samples + test_samples]

    val_files = files[train_samples + test_samples:train_samples + test_samples + val_samples]


    for filename in train_files:
        src_path = os.path.join(dataset_path, class_dir, filename)
        dest_path = os.path.join(train_output_path, class_dir, filename)
        os.makedirs(os.path.dirname(dest_path), exist_ok=True)
        shutil.copy(src_path, dest_path)
    for filename in test_files:
        src_path = os.path.join(dataset_path, class_dir, filename)
        dest_path = os.path.join(test_output_path, class_dir, filename)
        os.makedirs(os.path.dirname(dest_path), exist_ok=True)
        shutil.copy(src_path, dest_path)

    for filename in val_files:
        src_path = os.path.join(dataset_path, class_dir, filename)
        dest_path = os.path.join(val_output_path, class_dir, filename)
        os.makedirs(os.path.dirname(dest_path), exist_ok=True)
        shutil.copy(src_path, dest_path)


In [ ]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader, WeightedRandomSampler

# Define paths
train_output_path = "/content/drive/MyDrive/SC_190104027_Assignment1/Output/Train"
test_output_path = "/content/drive/MyDrive/SC_190104027_Assignment1/Output/Test"
val_output_path = "/content/drive/MyDrive/SC_190104027_Assignment1/Output/Validation"

# Define transformations for data preprocessing
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load datasets
train_dataset = ImageFolder(root=train_output_path, transform=transform)
test_dataset = ImageFolder(root=test_output_path, transform=transform)
val_dataset = ImageFolder(root=val_output_path, transform=transform)


# Create data loaders
batch_size =8  # Adjust as needed
train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=4,  # Set the number of workers for data loading
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

# Now you can use train_loader, test_loader, and val_loader for your training, testing, and validation respectively


In [ ]:
class_names = train_dataset.classes

In [ ]:
class_names

['COVID', 'NORMAL', 'PNEUMONIA']

In [ ]:
num_iters = 2000
input_dim = 256*256*3
output_dim = 3
learning_rate = 0.001
num_classes = 3

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [ ]:
# One Image Size
print(train_dataset[0][0].size())
#print(train_dataset[0][0].numpy().shape)
# First Image Label
#print(image_datasets['Train'][2000][1])

torch.Size([3, 224, 224])


In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, num_classes):
        super(NeuralNetwork, self).__init__()
        self.cnn_layer_1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1, padding=2)
        self.cnn_layer_2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        self.maxpool = nn.MaxPool2d(2, 2)
        self.flatten = nn.Flatten()
        self.linear_layer_1 = nn.Linear(32 * 56 * 56, 512)  # Correct input size
        self.linear_layer_2 = nn.Linear(512, 128)
        self.linear_layer_3 = nn.Linear(128, num_classes)
        self.tanh = nn.Tanh()  # Use Tanh activation function
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = self.cnn_layer_1(x)
        x = self.tanh(x)  # Apply Tanh activation
        x = self.maxpool(x)
        x = self.cnn_layer_2(x)
        x = self.tanh(x)  # Apply Tanh activation
        x = self.maxpool(x)
        x = self.flatten(x)
        x = self.linear_layer_1(x)
        x = self.dropout(x)
        x = self.tanh(x)  # Apply Tanh activation
        x = self.linear_layer_2(x)
        x = self.dropout(x)
        x = self.tanh(x)  # Apply Tanh activation
        x = self.linear_layer_3(x)
        return x


In [ ]:
model = NeuralNetwork(num_classes=num_classes)
model.to(device)

NeuralNetwork(
  (cnn_layer_1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (cnn_layer_2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_layer_1): Linear(in_features=100352, out_features=512, bias=True)
  (linear_layer_2): Linear(in_features=512, out_features=128, bias=True)
  (linear_layer_3): Linear(in_features=128, out_features=3, bias=True)
  (tanh): Tanh()
  (dropout): Dropout(p=0.2, inplace=False)
)

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
def trainModel(model, loss_fn, optimizer):
    model.train()
    total_loss = 0.0
    total=0
    correct = 0
    Ttotal=0
    Tcorrect=0
    TT=0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        # Get predictions from the maximum value
        _, predicted = torch.max(outputs, 1)

        # Total number of labels
        total += labels.size(0)


        # Total correct predictions
        if torch.cuda.is_available():
            correct += (predicted.cpu() == labels.cpu()).sum()
        else:
            correct += (predicted == labels).sum()

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        # Iterate through test dataset


    accuracy = 100 * correct.item() / total
    average_loss = total_loss / len(train_loader)
    print(f' Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}')

In [ ]:
for t in range(5):
    print(f"Epoch {t+1}\n-------------------------------")
    trainModel(model, criterion, optimizer)
print("Done!")

Epoch 1
-------------------------------


UnidentifiedImageError: ignored

In [ ]:
def testModel(model, loader):
    model.eval()
    total_loss = 0.0
    total = 0
    correct = 0

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum()

            total_loss += loss.item()

    accuracy = 100 * correct.item() / total
    average_loss = total_loss / len(loader)
    return average_loss, accuracy

# Test the model
test_loss, test_accuracy = testModel(model, test_loader)
print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}')

# Validate the model
val_loss, val_accuracy = testModel(model, val_loader)
print(f'Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}')

Test Loss: 0.1834, Test Accuracy: 93.3491
Validation Loss: 0.1982, Validation Accuracy: 92.6502


In [ ]:

# Calculate precision, recall, and F1-score
def calculate_metrics(loader):
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    precision = precision_score(all_labels, all_predictions, average='weighted')
    recall = recall_score(all_labels, all_predictions, average='weighted')
    f1 = f1_score(all_labels, all_predictions, average='weighted')
    return precision, recall, f1

test_precision, test_recall, test_f1 = calculate_metrics(test_loader)
val_precision, val_recall, val_f1 = calculate_metrics(val_loader)

print(f'Test Precision: {test_precision:.4f}, Recall: {test_recall:.4f}, F1-Score: {test_f1:.4f}')
print(f'Validation Precision: {val_precision:.4f}, Recall: {val_recall:.4f}, F1-Score: {val_f1:.4f}')

# Confusion matrix
def plot_confusion_matrix(loader):
    model.eval()
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    cm = confusion_matrix(all_labels, all_predictions)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.show()

plot_confusion_matrix(test_loader)
plot_confusion_matrix(val_loader)
